# Custom Context

`CognitiveContext` manages everything the agent needs to know: goal, tools, skills, and execution history. But real-world agents often need domain-specific data -- user preferences, constraints, knowledge bases, etc.

The **Exposure pattern** lets you add custom data fields to your context with two strategies:

- **`EntireExposure`** -- All information is always visible. The LLM sees the full summary every time. Use this for compact data that should always be in context (e.g., constraints, rules).
- **`LayeredExposure`** -- Progressive disclosure. The LLM sees a summary first and can request per-item details on demand. Use this for data where a summary suffices but details may be needed (e.g., preferences, knowledge items).

## 1. Setup

In [ ]:
import os
from typing import Any, Dict, List, Optional, Tuple
from pydantic import BaseModel, Field, ConfigDict

_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

from bridgic.core.cognitive import (
    CognitiveContext,
    CognitiveWorker,
    ThinkingMode,
    AgentAutoma,
    think_step,
    EntireExposure,
    LayeredExposure,
)
from bridgic.core.agentic.tool_specs import FunctionToolSpec
from bridgic.llms.openai import OpenAILlm

llm = OpenAILlm(api_base=_api_base, api_key=_api_key, timeout=120)


# --- Mock tools ---
def search_flights(origin: str, destination: str) -> str:
    """Search for available flights between two cities.

    Parameters
    ----------
    origin : str
        The departure city.
    destination : str
        The arrival city.
    """
    return f"Found 3 flights from {origin} to {destination}: FL100 ($300), FL200 ($450), FL300 ($280)"


def search_hotels(city: str) -> str:
    """Search for available hotels in a city.

    Parameters
    ----------
    city : str
        The city to search hotels in.
    """
    return f"Found 3 hotels in {city}: Grand Hotel ($120/night), City Inn ($80/night), Budget Stay ($50/night)"


def book_flight(flight_id: str) -> str:
    """Book a specific flight by its ID.

    Parameters
    ----------
    flight_id : str
        The flight identifier (e.g., FL100).
    """
    return f"Successfully booked flight {flight_id}. Confirmation: BK-{flight_id}-2024"


def book_hotel(hotel_name: str, nights: str) -> str:
    """Book a hotel for a specified number of nights.

    Parameters
    ----------
    hotel_name : str
        The name of the hotel to book.
    nights : str
        Number of nights to stay.
    """
    return f"Successfully booked {hotel_name} for {nights} nights. Confirmation: HBK-{hotel_name[:3].upper()}-2024"


travel_tools = [
    FunctionToolSpec.from_raw(search_flights),
    FunctionToolSpec.from_raw(search_hotels),
    FunctionToolSpec.from_raw(book_flight),
    FunctionToolSpec.from_raw(book_hotel),
]

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 2. Explore Built-in CognitiveContext

Before creating custom contexts, let's see what the built-in `CognitiveContext` provides. It has three Exposure fields:

- `tools` -- `CognitiveTools` (EntireExposure)
- `skills` -- `CognitiveSkills` (LayeredExposure)
- `cognitive_history` -- `CognitiveHistory` (LayeredExposure)

The `summary()` method returns a dictionary where each key maps to a formatted string. The `format_summary()` method joins them into a single string, with optional include/exclude filtering.

In [ ]:
from bridgic.core.cognitive import Step

# Create a basic context and add some data
ctx = CognitiveContext(goal="Plan a trip from Beijing to Kunming")

# Add tools
for tool_spec in travel_tools:
    ctx.tools.add(tool_spec)

# Add a history step
ctx.add_info(Step(
    content="Searched for flights from Beijing to Kunming",
    status=True,
    result="Found FL100, FL200, FL300",
    metadata={"tool_calls": ["search_flights"]}
))

# Print the context
print(ctx)

print(f'\n- - - - - summary() - - - - -')
for key, value in ctx.summary().items():
    print(f"[{key}]")
    print(f"  {value}")

print(f'\n- - - - - get_details() - - - - -')
detail = ctx.get_details("cognitive_history", 0)
print(f"cognitive_history[0] details: {detail}")

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 3. Custom `EntireExposure`

Let's create a custom `EntireExposure` field for **trip constraints** -- hard rules that the agent must always see (e.g., max budget, required departure date). Since these are always relevant, we use `EntireExposure` so the full information is included in every prompt.

You must implement `summary()` which returns a list of formatted strings.

In [ ]:
class Constraint(BaseModel):
    """A single trip constraint."""
    name: str
    value: str
    priority: str = "required"  # required, preferred


class TripConstraints(EntireExposure[Constraint]):
    """Trip constraints that are always fully visible to the agent."""

    def summary(self) -> List[str]:
        result = []
        for c in self._items:
            marker = "[REQUIRED]" if c.priority == "required" else "[PREFERRED]"
            result.append(f"{marker} {c.name}: {c.value}")
        return result


# Demo
constraints = TripConstraints()
constraints.add(Constraint(name="Max Budget", value="$500 total", priority="required"))
constraints.add(Constraint(name="Departure Date", value="Tomorrow", priority="required"))
constraints.add(Constraint(name="Airline Preference", value="No red-eye flights", priority="preferred"))

print("TripConstraints summary:")
for s in constraints.summary():
    print(f"  {s}")

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 4. Custom `LayeredExposure`

Now let's create a `LayeredExposure` field for **travel preferences** -- data where the agent sees a summary but can request details about specific items.

You must implement both `summary()` (brief overview) and `get_details(index)` (detailed information for a specific item).

In [ ]:
class Preference(BaseModel):
    """A single travel preference with summary and detailed info."""
    category: str
    brief: str  # Short description for summary
    details: str  # Full details available on request


class TravelPreferences(LayeredExposure[Preference]):
    """Travel preferences with progressive disclosure."""

    def summary(self) -> List[str]:
        return [f"{p.category}: {p.brief}" for p in self._items]

    def get_details(self, index: int) -> Optional[str]:
        if index < 0 or index >= len(self._items):
            return None
        p = self._items[index]
        return (
            f"Category: {p.category}\n"
            f"Summary: {p.brief}\n"
            f"Details: {p.details}"
        )


# Demo
prefs = TravelPreferences()
prefs.add(Preference(
    category="Budget",
    brief="Prefer budget-friendly options",
    details="Target $300 for flights, $50/night for hotels. Willing to go up to $400 for flights if significantly faster."
))
prefs.add(Preference(
    category="Accommodation",
    brief="Clean and central location",
    details="Must have WiFi and air conditioning. Prefer hotels within 2km of city center. No hostels or shared rooms."
))

print("TravelPreferences summary:")
for i, s in enumerate(prefs.summary()):
    print(f"  [{i}] {s}")

print(f"\nDetails for index 0:")
print(prefs.get_details(0))

print(f"\nDetails for index 1:")
print(prefs.get_details(1))

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 5. Custom Context Class

Now we combine everything into a custom `CognitiveContext` subclass. Custom Exposure fields are automatically detected by the framework -- just declare them as class attributes with the proper type annotations.

The `__post_init__` hook is the natural place to load tools and initialize custom data.

In [ ]:
class TravelPlanningContext(CognitiveContext):
    """Context with custom preferences and constraints."""
    model_config = ConfigDict(arbitrary_types_allowed=True)

    preferences: TravelPreferences = Field(default_factory=TravelPreferences)
    constraints: TripConstraints = Field(default_factory=TripConstraints)

    def __post_init__(self):
        # Load tools
        for tool_spec in travel_tools:
            self.tools.add(tool_spec)

        # Load constraints
        self.constraints.add(Constraint(name="Max Budget", value="$500 total", priority="required"))
        self.constraints.add(Constraint(name="Departure Date", value="Tomorrow", priority="required"))

        # Load preferences
        self.preferences.add(Preference(
            category="Budget",
            brief="Prefer budget-friendly options",
            details="Target $300 for flights, $50/night for hotels. Willing to go up to $400 for flights if significantly faster."
        ))
        self.preferences.add(Preference(
            category="Accommodation",
            brief="Clean and central location",
            details="Must have WiFi and air conditioning. Prefer hotels within 2km of city center. No hostels or shared rooms."
        ))

In [ ]:
# Create and inspect the custom context
ctx = TravelPlanningContext(goal="Plan a trip from Beijing to Kunming")

# Print the full context
print(ctx)

# Show format_summary() output
print(f'\n- - - - - format_summary() - - - - -')
print(ctx.format_summary())

# Show get_details() on custom fields
print(f'\n- - - - - get_details("preferences", 0) - - - - -')
print(ctx.get_details("preferences", 0))

# EntireExposure does NOT support get_details
print(f'\n- - - - - get_details("constraints", 0) - - - - -')
print(ctx.get_details("constraints", 0))  # Returns None

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 6. Integration with Agent

Custom context fields are automatically included in the prompt that the LLM sees. The agent can now make decisions based on the user's preferences and constraints -- and it can request details about specific preferences via the progressive disclosure mechanism.

In [ ]:
class BudgetAwareWorker(CognitiveWorker):
    """Worker that considers budget constraints and preferences."""

    async def thinking(self) -> str:
        return (
            "You are a budget-conscious travel assistant. "
            "Pay close attention to the constraints and preferences in the context. "
            "Always choose options that fit within the budget constraints. "
            "Plan ONE step at a time."
        )


class BudgetAwareAgent(AgentAutoma[TravelPlanningContext]):
    react = think_step(
        BudgetAwareWorker(llm=llm, mode=ThinkingMode.FAST, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        for _ in range(5):
            await self.react
            if ctx.finish:
                break


agent = BudgetAwareAgent(llm=llm)
result = await agent.arun(
    goal="I'm in Beijing and want to go to Kunming. Find me the cheapest flight and a budget hotel."
)

print(f'\n- - - - - Result - - - - -')
print(result)

## What have we learnt?

| Concept | Description |
|---------|-------------|
| `EntireExposure` | Always-visible data. Implement `summary()`. No per-item details. Best for compact, always-relevant data (constraints, rules). |
| `LayeredExposure` | Progressive disclosure. Implement `summary()` + `get_details(index)`. Best for data where summaries suffice but details may be needed (preferences, knowledge). |
| Custom Context | Extend `CognitiveContext` with custom Exposure fields. The framework auto-detects them and includes their summaries in the prompt. |
| `__post_init__` | Hook for loading tools, constraints, preferences, and other data when the context is created. |
| `get_details()` | Returns detail for `LayeredExposure` fields, `None` for `EntireExposure`. Disclosed details are persisted and included in subsequent prompts. |
| `format_summary()` | Joins all field summaries into a single string. Supports `include`/`exclude` filtering. |